# Question 1

In [1]:
import pandas as pd

df1 = pd.read_csv('CSV1.csv')
df2 = pd.read_csv('CSV2.csv')
df3 = pd.read_csv('CSV3.csv')
df4 = pd.read_csv('CSV4.csv')

print(df1.head())
print(df2.head())
print(df3.head())
print(df4.head())


   Unnamed: 0  HADM_ID                                         SHORT-TEXT  \
0           1   100003  history of present illness mr known lastname i...   
1           3   100009  history of present illness yo man with known c...   
2           5   100011  history of present illness y o male helmeted m...   
3           9   100021  history of present illness year old spanish sp...   
4          11   100028  history of present illness this is a year old ...   

                                           ICD9_CODE  \
0  53100, 2851, 07054, 5715, 45621, 53789, 4019, ...   
1  41401, 99604, 4142, 25000, 27800, V8535, 4148,...   
2  85206, 82111, 86403, 48242, 8600, 2851, 86121,...   
3  E8788, E8497, 4019, 04104, 0413, 5728, V1581, ...   
4  5761, 0389, 5184, 57481, 99591, 42731, 2875, 8...   

                                                ICD9    Label  
0        531, 285, 070, 571, 456, 537, 401, 535, 782      285  
1  414, 996, 414, 250, 278, V85, 414, 411, V45, V...      285  
2  852, 

Task 1

In [2]:
combined_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

combined_df.to_csv('combined_data.txt', sep='\t', index=False)

print("All CSV data extracted and saved to 'combined_data.txt'")

All CSV data extracted and saved to 'combined_data.txt'


Task 2

In [3]:
#pip install spacy

In [4]:
#pip install scispacy

In [5]:
#pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_sm-0.5.0.tar.gz

In [6]:
#pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz

In [7]:
#pip install transformers

In [8]:
#pip install torch 

In [9]:
#pip install datasets

In [10]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)



Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Task 3

Task 3.1

In [11]:
import collections
import csv
import re

with open('combined_data.txt', 'r') as file:
    text = file.read()

words = re.findall(r'\b\w+\b', text.lower())  

word_counts = collections.Counter(words)

top_30_words = word_counts.most_common(30)

with open('top_30_words.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Word', 'Count'])  
    writer.writerows(top_30_words)   

print("Top 30 most common words have been saved to 'top_30_words.csv'")


Top 30 most common words have been saved to 'top_30_words.csv'


Task 3.2

In [ ]:
from transformers import AutoTokenizer
from collections import Counter

def get_top_30_tokens(file_path, model_name="emilyalsentzer/Bio_ClinicalBERT", chunk_size=10000, max_seq_length=512):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    token_counts = Counter()
    
    with open(file_path, 'r', encoding='utf-8') as file:
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break 
            
            tokens = tokenizer.tokenize(chunk)

            for i in range(0, len(tokens), max_seq_length):
                token_chunk = tokens[i:i + max_seq_length]
                token_counts.update(token_chunk)

    top_30_tokens = token_counts.most_common(30)
    
    return top_30_tokens

top_30_tokens = get_top_30_tokens('combined_data.txt')
for token, count in top_30_tokens:
    print(f"Token: {token}, Count: {count}")


Task 4

In [ ]:
#pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_sm-0.5.0.tar.gz

In [ ]:
#pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz

In [ ]:
import spacy
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from collections import Counter
nlp_sci_sm = spacy.load("en_core_sci_sm")
nlp_bc5cdr = spacy.load("en_ner_bc5cdr_md")

nlp_sci_sm.max_length = 2_000_000 
nlp_bc5cdr.max_length = 2_000_000  


In [ ]:
nlp_sci_sm = spacy.load("en_core_sci_sm")     
nlp_bc5cdr = spacy.load("en_ner_bc5cdr_md") 

biobert_model = "dmis-lab/biobert-base-cased-v1.1"
tokenizer = AutoTokenizer.from_pretrained(biobert_model)
model = AutoModelForTokenClassification.from_pretrained(biobert_model)
biobert_ner = pipeline("ner", model=model, tokenizer=tokenizer)

with open('combined_data.txt', 'r') as file:
    text = file.read()

def extract_entities_scispacy(nlp_model, text, chunk_size=1000000):
    entities = {"diseases": [], "drugs": []}
    for i in range(0, len(text), chunk_size):
        chunk = text[i:i + chunk_size]
        doc = nlp_model(chunk)
        
        entities["diseases"].extend([ent.text for ent in doc.ents if ent.label_ == "DISEASE"])
        entities["drugs"].extend([ent.text for ent in doc.ents if ent.label_ == "CHEMICAL"])
    
    return entities["diseases"], entities["drugs"]

diseases_sci_sm, drugs_sci_sm = extract_entities_scispacy(nlp_sci_sm, text)
diseases_bc5cdr, drugs_bc5cdr = extract_entities_scispacy(nlp_bc5cdr, text)

def extract_entities_biobert(text):
    ner_results = biobert_ner(text)
    diseases = [res['word'] for res in ner_results if "DISEASE" in res['entity']]
    drugs = [res['word'] for res in ner_results if "CHEMICAL" in res['entity']]
    return diseases, drugs

diseases_biobert, drugs_biobert = extract_entities_biobert(text)

def count_entities(entities):
    return Counter(entities)

disease_count_sci_sm = count_entities(diseases_sci_sm)
drug_count_sci_sm = count_entities(drugs_sci_sm)

disease_count_bc5cdr = count_entities(diseases_bc5cdr)
drug_count_bc5cdr = count_entities(drugs_bc5cdr)

disease_count_biobert = count_entities(diseases_biobert)
drug_count_biobert = count_entities(drugs_biobert)

def compare_models(disease_count_model1, drug_count_model1, disease_count_model2, drug_count_model2):
    print("Total diseases detected (Model 1):", sum(disease_count_model1.values()))
    print("Total drugs detected (Model 1):", sum(drug_count_model1.values()))
    
    print("Total diseases detected (Model 2):", sum(disease_count_model2.values()))
    print("Total drugs detected (Model 2):", sum(drug_count_model2.values()))
    
    common_diseases = set(disease_count_model1.keys()).intersection(set(disease_count_model2.keys()))
    unique_to_model1 = set(disease_count_model1.keys()) - set(disease_count_model2.keys())
    unique_to_model2 = set(disease_count_model2.keys()) - set(disease_count_model1.keys())
    
    print("Common diseases:", len(common_diseases))
    print("Unique to Model 1:", len(unique_to_model1))
    print("Unique to Model 2:", len(unique_to_model2))
    
    common_drugs = set(drug_count_model1.keys()).intersection(set(drug_count_model2.keys()))
    unique_to_model1_drugs = set(drug_count_model1.keys()) - set(drug_count_model2.keys())
    unique_to_model2_drugs = set(drug_count_model2.keys()) - set(drug_count_model1.keys())
    
    print("Common drugs:", len(common_drugs))
    print("Unique drugs to Model 1:", len(unique_to_model1_drugs))
    print("Unique drugs to Model 2:", len(unique_to_model2_drugs))

print("\nComparing scispaCy (en_core_sci_sm) vs BioBERT")
compare_models(disease_count_sci_sm, drug_count_sci_sm, disease_count_biobert, drug_count_biobert)

print("\nComparing scispaCy (en_ner_bc5cdr_md) vs BioBERT")
compare_models(disease_count_bc5cdr, drug_count_bc5cdr, disease_count_biobert, drug_count_biobert)
